In this example, we'll combine numpy and ArcPy approaches to populate a field of ranked data.  This is complex to do in a field calculation, since it requires a global, rather than iterative, transformation.

In [1]:
# First, read in the necessary libraries

import arcpy
import numpy as np
from scipy import stats

In [8]:
# And define variables, including workspace, the input data, and the input and output fields.
# Additionally, define weather to provide a rank in ascending or descending order.
# Ascending means that higher values will receive higher (worse) rank values (1 to 1st, 10,000 to nth.)

arcpy.env.workspace = "data/"

layer = "ne_10m_populated_places_simple.shp"
field = "pop_max"
outfield = "npop_rank"
ascending = False

In [9]:
# Using a search cursor, iterate through the data, and record each value in a numpy array.

result = arcpy.GetCount_management(layer)
n_records = int(result.getOutput(0))
values = np.zeros(int(n_records),dtype=np.float)

with arcpy.da.SearchCursor(layer,field) as cursor:
    for i,row in enumerate(cursor):
        values[i] = row[0]
        
print(values)

[   21714.    21093.    23279. ...,  4630000.  5183700.  7206000.]


In [10]:
# If the field exists, delete the field before proceeding.
try:
    arcpy.DeleteField_management(layer,outfield)
    arcpy.AddMessage("Field exists. Deleting Field.")
except:
    arcpy.AddMessage("Field did not alrady exist.")
arcpy.AddMessage("Creating Field.")
arcpy.AddField_management(layer,outfield,"FLOAT")

<Result 'data\\ne_10m_populated_places_simple.shp'>

In [11]:
# Using scipy.stats, calculate the ranked data.  There are several ways to do this, and this could be
# an additional parameter.  Here, assign the average for tied ranks.
rank_values = stats.rankdata(values)
if ascending==False:
    rank_values = np.max(rank_values) + 1 - rank_values 
print(values)
print(rank_values)

[   21714.    21093.    23279. ...,  4630000.  5183700.  7206000.]
[ 5384.  5413.  5307. ...,    59.    50.    33.]


In [12]:
# Iterate through the field with an update cursor, writing the new values.

with arcpy.da.UpdateCursor(layer,outfield) as cursor:
    for i,row in enumerate(cursor):
        row[0] = rank_values[i]
        cursor.updateRow(row)

In [13]:
del cursor